# connect control system to diffractometer energy

This notebook is a work-in-progress.

```db

# file: registers.db
#
# purpose: test example for issue #72
# connect control system energy with diffractometer

record(ao, "EPICS:energy") {
  field(DESC, "monochromator energy")
  field(PREC, 5)
  field(EGU, "keV")
  field(VAL, 12.0)
}

record(ao, "EPICS:energy:offset") {
  field(DESC, "monochromator energy offset")
  field(PREC, 5)
  field(EGU, "keV")
  field(VAL, 0.0)
}

record(bo, "EPICS:energy:lock") {
  field(DESC, "lock hkl to mono?")
  field(ONAM, "locked")
  field(ZNAM, "unlocked")
  field(VAL, 0)
}
```

In [1]:
!caput EPICS:energy 12
!caput EPICS:energy.EGU keV
!caput EPICS:energy:offset 0
!caput EPICS:energy:lock locked

Old : EPICS:energy                   7.112
New : EPICS:energy                   12
Old : EPICS:energy.EGU               keV
New : EPICS:energy.EGU               keV
Old : EPICS:energy:offset            0
New : EPICS:energy:offset            0
Old : EPICS:energy:lock              locked
New : EPICS:energy:lock              locked


In [2]:
import gi
gi.require_version('Hkl', '5.0')
from hkl.geometries import E4CV
import logging
from ophyd import Component
from ophyd import EpicsMotor
from ophyd import EpicsSignal
from ophyd import PseudoSingle
import pint

logger = logging.getLogger(__name__)
ureg = pint.UnitRegistry()

In [3]:
class LocalDiffractometer(E4CV):
    h = Component(PseudoSingle, '', kind="hinted")
    k = Component(PseudoSingle, '', kind="hinted")
    l = Component(PseudoSingle, '', kind="hinted")

    omega = Component(EpicsMotor, "EPICS:m1", kind="hinted")
    chi = Component(EpicsMotor, "EPICS:m2", kind="hinted")
    phi = Component(EpicsMotor, "EPICS:m3", kind="hinted")
    tth = Component(EpicsMotor, "EPICS:m4", kind="hinted")

    energy = Component(EpicsSignal, "EPICS:energy")
    energy_EGU = Component(EpicsSignal, "EPICS:energy.EGU")
    energy_offset = Component(EpicsSignal, "EPICS:energy:offset")
    energy_update_calc_flag = Component(EpicsSignal, "EPICS:energy:lock")
    
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.energy_offset.subscribe(self._energy_offset_changed, event_type=EpicsSignal.SUB_VALUE)
#         self.energy_units.subscribe(self._energy_units_changed, event_type=EpicsSignal.SUB_VALUE)


In [4]:
fourc = LocalDiffractometer("", name="fourc")
fourc.wait_for_connection()
fourc._energy_changed()  # force the callback to update

In [5]:
def reporter(dfrct):
    print(f"{dfrct.calc.wavelength = }")
    print(f"{dfrct.calc.energy = }")
    print(f"{dfrct.energy.get() = }")
    print(f"{dfrct.energy_EGU.get() = }")
    print(f"{dfrct.energy_offset.get() = }")

reporter(fourc)

dfrct.calc.wavelength = 1.0332016666666666
dfrct.calc.energy = 12.0
dfrct.energy.get() = 12.0
dfrct.energy_EGU.get() = 'keV'
dfrct.energy_offset.get() = 0.0


In [6]:
!caput EPICS:energy 7.112

reporter(fourc) 

Old : EPICS:energy                   12
New : EPICS:energy                   7.112
dfrct.calc.wavelength = 1.7433098987626545
dfrct.calc.energy = 7.112
dfrct.energy.get() = 7.112
dfrct.energy_EGU.get() = 'keV'
dfrct.energy_offset.get() = 0.0
